In [1]:
from __future__ import annotations

from typing import TYPE_CHECKING, Any

import numpy as np
import igl
import meshplot as mp
import math

In [2]:
# Utility function to generate a tet grid
# n is a 3-tuple with the number of cell in every direction
# mmin/mmax are the grid bounding box corners
def tet_grid(n, mmin, mmax):
    nx = n[0]
    ny = n[1]
    nz = n[2]
    
    delta = mmax-mmin
    
    deltax = delta[0]/(nx-1)
    deltay = delta[1]/(ny-1)
    deltaz = delta[2]/(nz-1)
    
    T = np.zeros(((nx-1)*(ny-1)*(nz-1)*6, 4), dtype=np.int64)
    V = np.zeros((nx*ny*nz, 3))

    mapping = -np.ones((nx, ny, nz), dtype=np.int64)


    index = 0
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                mapping[i, j, k] = index
                V[index, :] = [i*deltax, j*deltay, k*deltaz]
                index += 1
    assert(index == V.shape[0])
    
    tets = np.array([
        [0,1,3,4],
        [5,2,6,7],
        [4,1,5,3],
        [4,3,7,5],
        [3,1,5,2],
        [2,3,7,5]
    ])
    
    index = 0
    for i in range(nx-1):
        for j in range(ny-1):
            for k in range(nz-1):
                indices = [
                    (i,   j,   k),
                    (i+1, j,   k),
                    (i+1, j+1, k),
                    (i,   j+1, k),

                    (i,   j,   k+1),
                    (i+1, j,   k+1),
                    (i+1, j+1, k+1),
                    (i,   j+1, k+1),
                ]
                
                for t in range(tets.shape[0]):
                    tmp = [mapping[indices[ii]] for ii in tets[t, :]]
                    T[index, :]=tmp
                    index += 1
                    
    assert(index == T.shape[0])
    
    V += mmin
    return V, T

# Reading point cloud

In [3]:
pi, v = igl.read_triangle_mesh("data/cat.off")
pi /= 10
ni = igl.per_vertex_normals(pi, v)
mp.plot(pi, shading={"point_size": 8})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

# Start of Assignment 

## Helper Functions

In [4]:
# TODO: DEFINE POINT
def find_closest_point_old(
    point: list[float, float, float],
    points: list[point]
) -> int: 
    """Returns the index of the closest point to the given point in the array."""
    dists = np.sum((points - point)**2, axis=1) 
    idx = np.unravel_index(
        np.where(dists != 0, dists, dists.max() + 1).argmin(),
        dists.shape
    )
    
    return idx[0]
    
def closest_points_old(
    point: list[float, float, float],
    points: list[point],
    d: float
) -> list[int]:
    """Returns the indices of the closest points to a given point that are less than a distance d away."""
    dists = np.sum((points - point)**2, axis=1)
    dists = np.sum((points - point)**2, axis=1) 
    indices = np.where(dists < d)
    # print( indices[0].tolist())
    return indices[0].tolist()
    

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
def find_closest_point(
    point: list[float, float, float],
    points: list[point],
    spatial_grid: list[point],
    unit_vectors: list[point],
    step
) -> int:
    """Returns the index of the closest point to the given point in the array taking spatial indexing into account"""
    
    # Variables
    closest_idx = 0
    min_value = 10000
    min_point = np.min(points, axis = 0)
    distance = np.array([math.floor(x) for x in (point - min_point) / step])

    # Get Buffer offset points
    check_distance = 1
    min_offset_x, min_offset_y, min_offset_z = [max(0, x) for x in (distance - check_distance).astype(int)]
    max_offset_x, max_offset_y, max_offset_z = [min(x, y)  for x, y in zip(distance + 1 + check_distance, unit_vectors)] 
    
    # Compute closest idx
    for i in range(min_offset_x, max_offset_x):
        for j in range(min_offset_y, max_offset_y):
            for k in range(min_offset_z, max_offset_z):
                idx = int(i + unit_vectors[0] * j + unit_vectors[0] * unit_vectors[1] * k)
                
                for q in range(len(spatial_grid[idx])):
                    d = np.linalg.norm(point - points[spatial_grid[idx][q]])
                    if d < min_value:
                        closest_idx = spatial_grid[idx][q]
                        min_value = d
                        
    return closest_idx


def closest_points(
    point: list[float, float, float,],
    points: list[point],
    h: float,
    spatial_grid: list[point],
    unit_vectors: list[point],
    step: float
) -> list[int]:
    """Returns the indices of the closest points to a given point that are less than a distance d away taking spatial indexing into account."""
    
    #Varaiables
    indices = list()
    min_point = np.min(points, axis = 0)
    original_points_count = int(points.shape[0] / 3)
    distance = np.array([math.floor(x) for x in (point - min_point) / step])
    
    # Get buffer offset points
    check_distance = 1
    min_offset_x, min_offset_y, min_offset_z = [max(0, x) for x in (distance - check_distance).astype(int)]
    max_offset_x, max_offset_y, max_offset_z = [min(x, y)  for x, y in zip(distance + 1 + check_distance, unit_vectors)] 
    
    # Get indices     
    for i in range(min_offset_x, max_offset_x):
        for j in range(min_offset_y, max_offset_y):
            for k in range(min_offset_z, max_offset_z):
                idx = int(i + unit_vectors[0] * j + unit_vectors[0] * unit_vectors[1] * k)
                
                for q in range(len(spatial_grid[idx])):
                    
                    distance = np.linalg.norm(point - points[spatial_grid[idx][q]])
                    if distance < h:
                        indices.append(spatial_grid[idx][q])
                        
                    distance = np.linalg.norm(point - points[original_points_count + spatial_grid[idx][q]])
                    if distance < h:
                        indices.append(original_points_count + spatial_grid[idx][q])
                        
                    distance = np.linalg.norm(point - points[original_points_count * 2 + spatial_grid[idx][q]])
                    if distance < h:
                        indices.append(original_points_count * 2 + spatial_grid[idx][q])
                    
    return indices
    

"""Gets the wendland weights"""
wendland = lambda r, h : pow((1 - (r / h)), 4) * ((4 * (r / h)) + 1) if (r < h) else 0

def create_poly(point: tuple[float, float, float], degree: int) -> Optional[list[int]]:
    """ Returns the basis given a point and degree """
    if degree == 0:
        return np.array([1])
    elif degree == 1:
        return np.array([1, point[0], point[1], point[2]])
    elif degree == 2:
        return np.array([
            1,
            grid_point[0],
            grid_point[1],
            grid_point[2],
            grid_point[0] * grid_point[1],
            grid_point[1] * grid_point[2],
            grid_point[0] * grid_point[2],
            grid_point[0] ** 2,
            grid_point[1] ** 2,
            grid_point[2] ** 2
        ])
    
    return None

## Spatial Index to Accelerate Calculations

In [5]:
def create_spatial_index(initial_step: float, p: list[Any]) -> Any:
    """ Creates a Spatial Index """
    min_point = np.amin(p, axis=0)
    max_point = np.amax(p, axis=0)
    dimension = 1.1 * (max_point - min_point)
    diagonal = np.linalg.norm(dimension)
    step = initial_step * diagonal
    
    unit_vectors = np.array([math.ceil(x) for x in dimension / step])
    
    spatial_grid = [[] for i in range(int(np.prod(unit_vectors)))]
    
    for i in range(len(p)):
        distance = np.array([math.floor(x) for x in (p[i] - min_point) / step])
        idx = int(distance[0] + unit_vectors[0] * distance[1] + unit_vectors[0] * unit_vectors[1] * distance[2])
        
        spatial_grid[idx].append(i)
    
    return spatial_grid, unit_vectors, step

## Setting up the Constraints

In [6]:
def get_constraints(pi, ni, spatial_grid, unit_vectors, step):
    """Sets up the the set of constraint equations by choosing constraint locations and values.
    
    
    - For each point pi in the point cloud, add a constraint of the form f(pi) = di = 0.
    - Fix an eps value, for instance eps = 0.01 bounding_box_diagonal.
    - For each point pi compute pi+ = pi + eps ni, where ni is the normalized normal of pi. Check that pi is the 
      closest point to pi+ if not, halve esp and recompute pi+ until this is the case. Then, add another 
      constraint equation: f(pi+) = eps.
    - Repeat the same process for -eps, i.e., add equations of the form f(pi-) = -eps. Do not forget to check 
      each time that pi is the closest point to pi-.
    - Append the tree vectors pi, pi+, and pi- and corresponding f to a unique vector p and f.
    """
    p = np.zeros((pi.shape[0] * 3, 3))
    f = np.zeros((pi.shape[0] * 3))

    diagonal = igl.bounding_box_diagonal(pi)
    epsilon = 0.01 * diagonal
    size = len(pi)

    for i in range(size):
        p[i] = pi[i]
        f[i] = 0
        
        # Get positive epsilon values    
        p[i + size] = pi[i] + epsilon * ni[i]    
        while find_closest_point(pi[i] + epsilon * ni[i], pi, spatial_grid, unit_vectors, step) != i:
            epsilon *= 0.5
            
        p[i + size] = pi[i] + epsilon * ni[i]
        f[i + size] = epsilon

        # Get negative epsilon values    
        epsilon = 0.01 * diagonal
        p[i + 2 * size] = pi[i] - epsilon * ni[i]       
        while find_closest_point(pi[i] - epsilon * ni[i], pi, spatial_grid, unit_vectors, step) != i:
            eps *= 0.5
            
        p[i + 2 * size] = pi[i] - epsilon * ni[i]
        f[i + 2 * size] = -epsilon 
    
    return p, f

In [7]:
# Variables
bbox_min = np.amin(pi, axis=0)
bbox_max = np.amax(pi, axis=0)
bbox_diagonal = np.linalg.norm(bbox_max - bbox_min)
epsilon = 0.01 * bbox_diagonal
initial_step = 0.1

spatial_grid, unit_vectors, step = create_spatial_index(initial_step, pi)
p, f = get_constraints(pi, ni, spatial_grid, unit_vectors, step)

colors = np.zeros((pi.shape[0]*3, 3))
for i in range(pi.shape[0]):
    colors[i] = np.array([1., 0., 0.])
    colors[i + pi.shape[0]] = np.array([0., 1., 0.])
    colors[i + 2 * pi.shape[0]] = np.array([0., 0., 1.])
    

mp.plot(p, c=colors, shading={"point_size": 8})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0120944…

## MLS Interpolation

In [8]:
def evaluate_function(
    point: list[float, float, float],
    p: list[point],
    f: list[point],
    spatial_grid: list[point],
    unit_vectors: list[point],
    degree: int,
    radius: float,
    step: float
) -> None:
    """ Constructs and evaluates an implicit function satisfying the constraints as nearly as possible. """
    # Variable
    indices = closest_points(point, p, radius, spatial_grid, unit_vectors, step)
    indices_size = len(indices)
    num_poly = 3 ** degree + 1
    
    # Note if the number of constraint points within wendlandRadius is less than twice the number 
    # of polynomial coefficients (i.e., 1 for k = 0, 4 for k = 1, and 10 for k = 2), you can 
    # assign a large positive (outside) value to the grid point.
    if poly_degree == 0:
        num_poly = 1
    
    if(indices_size < num_poly * 2):
        return 10000
    else:
        B = np.zeros((indices_size, num_poly))
        W = np.eye(indices_size)
        constrained_point = p[indices]
        constrained_value = f[indices]
        
        for i in range(indices_size):
            B[i] = create_poly(constrained_point[i], degree)
            W[i, i] = wendland(np.linalg.norm(point - constrained_point[i], 2), radius)

        ax = np.linalg.solve(((B.T).dot(W)).dot(B), ((B.T).dot(W)).dot(constrained_value))
        return create_poly(point, degree).dot(ax)    

In [9]:
resolution = 15
x, T = tet_grid(
    (resolution, resolution, resolution),
    bbox_min - 0.05 * bbox_diagonal,
    bbox_max + 0.05 * bbox_diagonal
)

poly_degree = 1
wendland_radius = bbox_diagonal * 0.1
distance = 20

fx = np.array([
    evaluate_function(
        point, p, f, spatial_grid, unit_vectors, poly_degree, wendland_radius, step
    ) for point in x
])

colors = np.zeros_like(fx)
colors[fx < 0] = -1
colors[fx >= 0] = 1

mp.plot(x, c=colors, shading={"point_size": 6, "width": 1000, "height": 1000})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(5.0, -23.…

## Extracting the Surface

In [11]:
def marching_tet(
    x: list[float, float, float],
    T: Any,
    fx: list[point]
) -> tuple[Any, Any]:
    """ Use marching tets to extract the zero isosurface from grid. """
    sv, sf, _, _ = igl.marching_tets(x, T, fx, 0)

    # Generate face components
    f = igl.face_components(sf)
    components = list()
    for i in range(f.shape[0]):
        if f[i] not in components:
            components.append(f[i])
    
    connected_components = np.zeros(len(components))
    
    for i in range(f.shape[0]):
        connected_components[f[i]] += 1
    
    return sv, sf[f == np.argmax(connected_components)]


sv, new_sf = marching_tet(x, T, fx)
mp.plot(sv, new_sf, shading={"wireframe": True}) 

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(6.2883472…

## Output for Extracting the Surface

In [12]:
def extract_surface_full(datapath, resolution, step, degree, radius_rate) -> None:
    """ Quick Acess function to quickly render the surface for diff values """
    bbox_min = np.amin(pi, axis=0)
    bbox_max = np.amax(pi, axis=0)
    bbox_diagonal = np.linalg.norm(bbox_max - bbox_min)
    epsilon = 0.01 * bbox_diagonal
    initial_step = 0.1

    spatial_grid, unit_vectors, step = create_spatial_index(initial_step, pi)
    p, f = get_constraints(pi, ni, spatial_grid, unit_vectors, step)

    x, T = tet_grid(
        (resolution, resolution, resolution),
        bbox_min - 0.05 * bbox_diagonal,
        bbox_max + 0.05 * bbox_diagonal
    )

    wendland_radius = bbox_diagonal * radius_rate
    distance = 20

    fx = np.array([
        evaluate_function(
            point, p, f, spatial_grid, unit_vectors, poly_degree, wendland_radius, step
        ) for point in x
    ])

    marching_tet(x, T, fx)
    mp.plot(sv, new_sf, shading={"wireframe": True}) 
    

In [13]:
extract_surface_full("data/cat.off", 20, 0.06, degree = 1, radius_rate = 0.1)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(6.2883472…

In [14]:
extract_surface_full("data/cat.off", 20, 0.06, degree = 1, radius_rate = 0.1)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(6.2883472…

In [15]:
extract_surface_full("data/cat.off", 40, 0.05, degree = 1, radius_rate = 0.3)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(6.2883472…

In [16]:
extract_surface_full("data/cat.off", 40, 0.01, degree = 2, radius_rate = 0.1)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(6.2883472…